# Trabalho de AEDV (A1)
    Igor Cortes Junqueira
    Igor Patricio Michels

Preparação do ambiente:

In [74]:
library(readr)
library(stringr)
library(dplyr)
shows <- read_csv("tv_shows.csv")
disney <- read_csv("disney_plus_shows.csv")
netflix <- read_csv("netflix_titles.csv")
prime <- read_csv("Prime TV Shows Data set.csv")


── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Title = col_character(),
  Year = col_double(),
  Age = col_character(),
  IMDb = col_double(),
  Rotten_Tomatoes = col_character(),
  Netflix = col_double(),
  Hulu = col_double(),
  Prime_Video = col_double(),
  DisneyPlus = col_double(),
  type = col_double()
)


Warning message:
“85 parsing failures.
row col   expected    actual           file
 63  -- 11 columns 1 columns 'tv_shows.csv'
 69  -- 11 columns 1 columns 'tv_shows.csv'
423  -- 11 columns 1 columns 'tv_shows.csv'
453  -- 11 columns 1 columns 'tv_shows.csv'
528  -- 11 columns 1 columns 'tv_shows.csv'
... ... .......... ......... ..............
See problems(...) for more details.
”

── Column specification ────────────────────────────────────────────────────────
cols(
  imdb_id = col_character(),
  title = col_character(),
  plot = col_character(),
  type = col_character(),
  rated = col_character(),
  year = co

Compondo a base do trabalho:

In [97]:
cshows <- shows %>% mutate(Streamings=paste(sep=" ", ifelse(Netflix==1,"Netflix",""),
                                                ifelse(Prime_Video==1,"Prime",""),
                                                ifelse(DisneyPlus==1,"Disney","")) %>%
                                      trimws() %>% str_replace(" ",", ")) %>%
                    select(-Netflix,-Hulu,-Prime_Video,-DisneyPlus,-type)

cshows

ID,Title,Year,Age,IMDb,Rotten_Tomatoes,Streamings
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
0,Breaking Bad,2008,18+,9.5,96%,Netflix
1,Stranger Things,2016,16+,8.8,93%,Netflix
2,Money Heist,2017,18+,8.4,91%,Netflix
3,Sherlock,2010,16+,9.1,78%,Netflix
4,Better Call Saul,2015,18+,8.7,97%,Netflix
5,The Office,2005,16+,8.9,81%,Netflix
6,Black Mirror,2011,18+,8.8,83%,Netflix
7,Supernatural,2005,16+,8.4,93%,Netflix
8,Peaky Blinders,2013,18+,8.8,92%,Netflix
